## 2D Convex Hull

In [ ]:
import random
import matplotlib.pyplot as plt

def generate_random_points(xx=[-50, 50], yy=[-50, 50], num_points=50):
        """
        生成指定数量的随机坐标点集合。
        
        参数:
        xx (list|tuple): 横坐标取值范围
        yy (list|tuple): 纵坐标取值范围
        num_points (int): 需要生成的坐标点的数量。
        
        返回:
        list: 包含随机坐标点的列表，每个点是一个元组 (x, y)
        """
        x_min, x_max = xx
        y_min, y_max = yy
        points = []
        for _ in range(num_points):
                x = random.randint(x_min, x_max)
                y = random.randint(y_min, y_max)
                points.append((x, y))
        return points

def plot_points(points, poly_points=None, plot_polygon=False):
        """
        将给定的点集绘制在图上，并可选地绘制一个多边形。
        
        参数:
        points (list): 包含坐标点的列表，每个点是一个元组 (x, y)。
        poly_points (list, optional): 包含坐标点的列表，用于绘制封闭多边形，默认为 None。
        """
        # 绘制随机点
        x_values = [point[0] for point in points]
        y_values = [point[1] for point in points]
        plt.scatter(x_values, y_values, color='blue', label='random points')

        # 绘制多边形
        if poly_points:    # poly_points不为None和[]
                poly_x_values = []
                poly_y_values = []

                for point in poly_points:
                        poly_x_values.append(point[0])
                        poly_y_values.append(point[1])

                # 将最后一个点与第一个点相连，形成封闭多边形
                poly_x_values.append(poly_x_values[0])
                poly_y_values.append(poly_y_values[0])

                plt.scatter(poly_x_values, poly_y_values, color='red', marker='x', label='extreme points')
                if plot_polygon:
                        plt.plot(poly_x_values, poly_y_values, color='red', label='extreme edges')

        plt.title('Convex Hull')
        plt.xlabel('X-Axis')
        plt.ylabel('Y-Axis')
        plt.legend(bbox_to_anchor=(1.05, 1.2), loc='upper right')
        plt.grid(True)
        plt.show()
        
        
random_points = generate_random_points()
print(random_points)
# 示例多边形点集
poly_points = [(-50, -50), (50, -50), (50, 50), (-50, 50)]
plot_points(random_points, poly_points)

### Extreme edges algorithms

In [ ]:
from ComputerGraphics.convex_hull_2d import *

def check_is_extreme_edge(points, line) -> bool:
        """判断line是否为极值边"""
        p1, p2 = line
        p1_p2 = np.array(p2) - np.array(p1)
        cross_arr = []
        for p in points:
                if p != p1 and p != p2:
                        p1_p = np.array(p) - np.array(p1)
                        cross_arr.append(int(np.cross(p1_p2, p1_p)))
        return (np.array(cross_arr) <= 0).all() or (np.array(cross_arr) >= 0).all()
        
@timer
def extreme_edged_algo(points) -> list[tuple]:
        """极值边算法，O(n^3)"""
        res_points = set()
        for p1 in points:
                for p2 in points:
                        if p1 == p2: continue
                        if check_is_extreme_edge(points, [p1, p2]):
                                res_points.add(p1)
                                res_points.add(p2)
        return list(res_points)

points = generate_random_points()
hull = extreme_edged_algo(points)
plot_points(points, hull, True)

### Gift Wrapping Algorithm

先找到最右下角点，然后逐个逆时针找下一个角度最小的点

In [ ]:
from ComputerGraphics.convex_hull_2d import *

def orientation(p, q, r):
        """计算三个点的方向：
             0 -> p, q 和 r 共线
             1 -> 顺时针
             2 -> 逆时针
        """
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
                return 0
        return 1 if val > 0 else 2

@timer
def gift_wrapping_algo(points):
        """使用 Gift Wrapping Algorithm 计算凸包"""
        if len(points) < 3:
                return points    # 凸包至少需要 3 个点

        # 找到最右下角的点
        rightmost = max(points, key=lambda p: (p[0], p[1]))
        convex_hull = []

        p = rightmost
        while True:
                convex_hull.append(p)
                # 在当前点 p 的下一个点 q 之前，选择所有其他点 r
                q = points[0]
                for r in points[1:]:
                        # 选择 q 为与 p 的连接线形成顺时针方向的点
                        if (q == p) or (orientation(p, q, r) == 1):    # 顺时针方向
                                q = r
                p = q
                # 如果下一个点是最右下角的点，结束循环
                if p == rightmost:
                        break

        return convex_hull

points = generate_random_points()
hull = gift_wrapping_algo(points)
plot_points(points, hull, True)

### Quick Hull Algorithm

1. 找到平面上最左边和最右边的点，这两点必然属于凸包。
2. 将点集分成两部分，一部分位于这两点连线的上方，另一部分位于下方。
3. 对每一部分递归进行处理：
     1. 对每部分找到离线段最远的点。
     2. 这个最远的点与线段两端构成新的两条线段，分别递归地继续寻找更多的凸包边界上的点。

In [ ]:
from ComputerGraphics.convex_hull_2d import *

def distance_p2l(point, line):
        """计算点 point 到直线 line 的垂直距离"""
        p, q = line
        return abs((q[1] - p[1]) * (point[0] - p[0]) - (q[0] - p[0]) * (point[1] - p[1]))

def find_side(p, q, r):
        """确定点 r 相对于线段 pq 的位置:
             返回正数 -> r 在pq的左边
             返回负数 -> r 在pq的右边
             返回0 -> r 在pq上
        """
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val > 0:
                return 1
        elif val < 0:
                return -1
        else:
                return 0

@timer
def quick_hull_algo(points) -> list[tuple]:
        """主函数，计算点集的凸包"""
        if len(points) < 3:
                return points
        
        def quick_hull(points, p, q, side):
                """递归寻找凸包上的点"""
                idx = -1
                max_dist = 0

                # 寻找点集中离线段 pq 距离最远的点
                for i in range(len(points)):
                        temp = distance_p2l(points[i], [p, q])
                        if find_side(p, q, points[i]) == side and temp > max_dist:
                                idx = i
                                max_dist = temp

                # 如果没有外侧点，则 p-q 是凸包的一条边
                if idx == -1:
                        return [p, q]

                # 递归求解：找离 p-r 和 r-q 两条边最远的点
                chull = []
                chull.extend(quick_hull(points, points[idx], p, -find_side(points[idx], p, q)))
                chull.extend(quick_hull(points, points[idx], q, -find_side(points[idx], q, p)))
                return chull
        
        # 找到最左边和最右边的点
        min_x = min(points, key=lambda p: p[0])
        max_x = max(points, key=lambda p: p[0])

        # 上方点集和下方点集分别计算
        upper_hull = quick_hull(points, min_x, max_x, 1)
        lower_hull = quick_hull(points, min_x, max_x, -1)

        # 合并上下凸包，去重
        convex_hull = upper_hull + lower_hull
        unique_hull = []
        for pt in convex_hull:
                if pt not in unique_hull:
                        unique_hull.append(pt)

        return unique_hull

# 输入点集
points = generate_random_points()
# 计算凸包
hull = quick_hull_algo(points)
plot_points(points, hull, True)

### Graham Algorithm

1. 选择一个基准点（通常是最下方最左侧的点）。
2. 将其余点按与基准点形成的极角排序。
3. 使用栈维护构造的凸包边界，遍历排序后的点集并依次检查是否符合凸包要求（即是否逆时针转向）。

In [ ]:
def orientation(p, q, r):
        """计算三个点的方向：
             0 -> p, q 和 r 共线；
             1 -> 顺时针；
             2 -> 逆时针
        """
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
                return 0
        return 1 if val > 0 else 2

p = (0, 0)
q = (1, 1)
r = (1, 2)
orientation(p, q, r)

In [ ]:
from ComputerGraphics.convex_hull_2d import *

def center(points) -> tuple:
        """ 计算一组点的中心点（质心） """
        # 初始化各维度的和
        sum_x = sum_y = 0
        
        # 计算每个维度的总和
        for point in points:
                sum_x += point[0]
                sum_y += point[1]
        
        # 计算平均值
        num_points = len(points)
        avg_x = sum_x / num_points
        avg_y = sum_y / num_points
        return (avg_x, avg_y)

# 函数1：计算两个点之间的极角
def polar_angle(p0, p1):
        """返回点 p1 相对于点 p0 的极角"""
        return math.atan2(p1[1] - p0[1], p1[0] - p0[0])

def distance_p2p(p0, p1):
        """ 计算两点 p0, p1 之间的欧几里得距离 """
        return math.sqrt((p1[0] - p0[0]) ** 2 + (p1[1] - p0[1]) ** 2)

@timer
def graham_scan_algo(points):
        """ 使用 Graham Scan Algorithm 算法求凸包 """
        # 第一步：找到最下方的点，如果有多个则选择最左边的点
        start = min(points, key=lambda p: (p[1], p[0]))

        # 第二步：按与基准点的极角进行排序
        sorted_points = sorted(points, key=lambda p: (polar_angle(start, p), -p[1], p[0]))

        # 第三步：构建凸包，使用栈保存当前的边界
        hull = []
        for p in sorted_points:
                while len(hull) > 1 and orientation(hull[-2], hull[-1], p) != 2:
                        hull.pop()    # 如果不是逆时针，则移除最后的点
                hull.append(p)

        return hull


points = generate_random_points()
hull = graham_scan_algo(points)
plot_points(points, hull, True)

### Incremental Algorithm

从一个初始的凸包开始（通常是最开始的两个点），对于每个新点，判断它与当前凸包的关系，并更新凸包。

1. 初始化凸包：从输入点集中选择初始的两个点（最右上角的两个点）作为凸包的边界。
2. 逐个添加点：对于每个新的点，检查它与当前凸包的各个边的关系。
3. 更新凸包：如果新点在当前凸包外侧，则需要找到新点与当前边相交的点，并调整凸包。

In [ ]:
from enum import Enum

def cross_product(v1: list[tuple], v2: list[tuple]):
        """ 计算向量v1和向量v2的叉积 """
        return np.cross(np.array(v1[1]) - np.array(v1[0]), np.array(v2[1]) - np.array(v2[0]))

class Orientation(Enum):
        CLOCKWISE = 1
        ANTI_CLOCKWISE = 2
        COLLINEATION = 0
        
def orientation(p, q, r):
        """ 计算 p, q, r 三个点的方向 """
        val = cross_product([p, q], [q, r])
        if val == 0:
                return Orientation.COLLINEATION
        return Orientation.ANTI_CLOCKWISE if val > 0 else Orientation.CLOCKWISE

orientation((0, 0), (1, 1), (1, 2))

In [ ]:
from ComputerGraphics.convex_hull_2d import *

@timer
def incremental_algo(points):
        """ Incremental Algorithm 算法求凸包 """
        points = sorted(points, key=lambda x: (x[0], x[1]))    # 将点按x坐标排序，如果x相同则按y坐标排序
        if len(points) < 3:
                return points    # 点数少于3个，直接返回这些点
        lower_hull = []
        for p in points:
                # 保证每次加入的点都保持凸性，删除不满足条件的点
                while len(lower_hull) >= 2 and cross_product([lower_hull[-2], lower_hull[-1]], [lower_hull[-2], p]) <= 0:
                        lower_hull.pop()
                lower_hull.append(p)

        upper_hull = []
        for p in reversed(points):
                while len(upper_hull) >= 2 and cross_product([upper_hull[-2], upper_hull[-1]], [upper_hull[-2], p]) <= 0:
                        upper_hull.pop()
                upper_hull.append(p)

        # 合并上半部分和下半部分，最后一个点重复，所以去掉一个
        return lower_hull[:-1] + upper_hull[:-1]

points = generate_random_points()

# 计算凸包
hull = incremental_algo(points)

# 输出并绘制凸包
print("凸包的点集:", hull)
plot_points(points, hull, True)

### Divide and conquer Algorithm

算法思路类似于归并排序，先将点集分成两半，然后递归地计算每一半（左右两半）的凸包，递归至只剩下3个点时形成最小凸包，最后逐步合并两个凸包，得到完整的凸包

1. 分割：将点集按 x 坐标排序，然后将其分为左右两部分。
2. 递归：分别计算左右两部分的凸包。
3. 合并：将左右部分的凸包合并，处理跨越分割线的点，并找出最终的凸包。

In [ ]:
from ComputerGraphics.convex_hull_2d import generate_random_points, plot_points

def cross_product(o, a, b):
        """计算向量OA和OB的叉积"""
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

def merge_hulls(left_hull, right_hull):
        """合并左右两个凸包"""
        # 从左凸包找到最右侧的点
        left_most = left_hull[-1]
        right_most = right_hull[0]

        # 从左侧向右寻找上半边界
        upper = []
        upper.append(left_most)
        i = left_hull.index(left_most) - 1
        j = right_hull.index(right_most)
        while True:
                upper.append(right_hull[j])
                if cross_product(left_most, upper[-2], right_hull[j]) > 0:
                        break
                j = (j + 1) % len(right_hull)

        # 从右侧向左寻找下半边界
        lower = []
        lower.append(left_most)
        i = left_hull.index(left_most) - 1
        j = right_hull.index(right_most)
        while True:
                lower.append(right_hull[j])
                if cross_product(left_most, lower[-2], right_hull[j]) < 0:
                        break
                j = (j - 1) % len(right_hull)

        return upper + lower

# 函数3：递归计算凸包
def divide_and_conquer(points):
        """分治法计算凸包"""
        if len(points) < 3:
                return points

        # 按照x坐标排序
        points = sorted(points, key=lambda p: p[0])

        # 分割点集
        mid = len(points) // 2
        left_hull = divide_and_conquer(points[:mid])
        right_hull = divide_and_conquer(points[mid:])

        # 合并左右凸包
        return merge_hulls(left_hull, right_hull)

points = [(0, 3), (1, 1), (2, 2), (4, 4), (0, 0), (1, 2), (3, 1), (3, 3)]
hull = divide_and_conquer(points)
plot_points(points, hull)